In [ ]:
import random, os, time, math, re

import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# # Detect hardware, return appropriate distribution strategy
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is
#     # set: this is always the case on Kaggle.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
#     strategy = tf.distribute.get_strategy()

# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train1              = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
# train1              = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv")
# train1                = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
# train1              = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv")
                          
# sample_submission   = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv")
# validation          = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv")
# test                = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv")

# validation2         = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv")
# test2               = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv")    

In [ ]:
train1.describe()

In [ ]:
len(train1)

In [ ]:
train=train1[["comment_text","toxic"]][0:1000]

In [ ]:
pd.set_option('display.max_colwidth', None)
train.head(50)

## Preprocessing

In [ ]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 

In [ ]:
train["toxic"]= (train["toxic"] > 0.4).astype(int)

In [ ]:
train

In [ ]:
ax = sns.barplot( data=train)

### remove stop words and punctuation

In [ ]:
punctuations = string.punctuation
stop_words = set(stopwords.words("english"))

def remove_stop_words(words):
    return [w for w in words if not w in stop_words]

def remove_empty(words):
    return [w for w in words if not len(w.strip())<2]

def words_to_lowercase(words):
    return [w.lower() for w in words]

def remove_punctuation_from_word(word):
    return ''.join(c for c in word if c not in punctuations)

def remove_punctuation_from_words(words):
    return [remove_punctuation_from_word(w) for w in words]

In [ ]:
train_cleaned = train.copy()

train_cleaned['comment_text'] = train['comment_text'].apply(word_tokenize).apply(words_to_lowercase).apply(remove_stop_words).apply(remove_punctuation_from_words).apply(remove_empty)

In [ ]:
print(train['comment_text'][0:3])

In [ ]:
print(train_cleaned['comment_text'][0:3])

Detection of emojis


In [ ]:
#Installing emot library
!pip install emot
#Importing libraries
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

In [ ]:
# Function for converting emojis into word
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text
# Example
text1 = "Hilarious 😂. The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒"
convert_emojis(text1)

Detection of abreviation 


In [ ]:
pip install scispacy


In [ ]:
from scispacy.abbreviation import AbbreviationDetector

In [ ]:
abbreviation_pipe=AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)
def replace_acronyms(text):
    doc=nlp(txt)
    altered_tok=[tok.text for tok in doc]
    print(doc._.abbreviations)
    for abrv in doc._.abbreviations:
        altered_tok[abrv.start]=str(abrv._.long_form)
    return(" "join(altered_tok))


### stemming

In [ ]:
ps = PorterStemmer() 

def stem_words(words):
    return [ps.stem(w) for w in words]

In [ ]:
print(train_cleaned['comment_text'][0:3])

In [ ]:
#train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(stem_words)

In [ ]:
#print(train_cleaned['comment_text'][0:3])

### Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer() 

def lemmatize_words(words):
    return [lemmatizer.lemmatize(w) for w in words]

In [ ]:
print(train_cleaned['comment_text'][0:3])

In [ ]:
train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(lemmatize_words)

In [ ]:
print(train_cleaned['comment_text'][0:3])

## Vectorisation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(" ".join)

In [ ]:
train_cleaned.head()

In [ ]:
print(train_cleaned['comment_text'][0:3])

### Count vectorization

In [ ]:
count_vectorizer = CountVectorizer() 
count_vectorizer.fit(train_cleaned['comment_text']) 

In [ ]:
print("Vocabulary: ", count_vectorizer.vocabulary_) 

In [ ]:
 count_vector = count_vectorizer.transform(train_cleaned['comment_text']) 

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
tfidf_vector = tfidf_vectorizer.fit_transform(train_cleaned['comment_text'])

In [ ]:
print("Vocabulary: ", tfidf_vectorizer.vocabulary_) 

> ## Prediction

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [ ]:
clf_count = MultinomialNB()
clf_tfidf = MultinomialNB()

### Prediction using Count vectorization

In [ ]:
x_df = count_vector
y_df = train_cleaned.toxic

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.10, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_regression = LogisticRegression().fit(x_train, y_train)

In [ ]:
clf_regression.score(x_test, y_test)

In [ ]:
clf_count.fit(x_train, y_train)

In [ ]:
clf_count.score(x_test, y_test)

In [ ]:
toxic_comment_example = "haha you are losers"
non_toxic_comment_example = "thank you for your response"

toxic_comment_example_vect = count_vectorizer.transform([toxic_comment_example]).toarray()
non_toxic_comment_example_vect = count_vectorizer.transform([non_toxic_comment_example]).toarray()

toxic_comment_example_prediction = clf_count.predict(toxic_comment_example_vect)
non_toxic_comment_example_prediction = clf_count.predict(non_toxic_comment_example_vect)

print("prediction for toxic comment example : ",toxic_comment_example_prediction[0])
print("prediction for non toxic comment example : ",non_toxic_comment_example_prediction[0])

### Prediction using TF-IDF

In [ ]:
x_df = tfidf_vector
y_df = train_cleaned.toxic

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.10, random_state=42)

In [ ]:
clf_tfidf.fit(x_train, y_train)

In [ ]:
clf_tfidf.score(x_test, y_test)

In [ ]:
toxic_comment_example = "haha you are losers"
non_toxic_comment_example = "thank you for your response"

toxic_comment_example_vect = tfidf_vectorizer.transform([toxic_comment_example])
non_toxic_comment_example_vect = tfidf_vectorizer.transform([non_toxic_comment_example])

toxic_comment_example_prediction = clf_tfidf.predict(toxic_comment_example_vect)
non_toxic_comment_example_prediction = clf_tfidf.predict(non_toxic_comment_example_vect)

print("prediction for toxic comment example : ",toxic_comment_example_prediction[0])
print("prediction for non toxic comment example : ",non_toxic_comment_example_prediction[0])

In [ ]:
toxic_comment_example = "haha you are losers"
non_toxic_comment_example = "thank you for your response"

toxic_comment_example_vect = count_vectorizer.transform([toxic_comment_example]).toarray()
non_toxic_comment_example_vect = count_vectorizer.transform([non_toxic_comment_example]).toarray()

toxic_comment_example_prediction = clf_regression.predict(toxic_comment_example_vect)
non_toxic_comment_example_prediction = clf_regression.predict(non_toxic_comment_example_vect)

print("prediction for toxic comment example : ",toxic_comment_example_prediction[0])
print("prediction for non toxic comment example : ",non_toxic_comment_example_prediction[0])

In [ ]:
pip install transformers